In [1]:
using JLD2
using Flux

In [2]:
@load "Training.jld2" inputs;


In [3]:
@load "Targets.jld2"  actuals;

In [4]:
input = inputs[1,:];

In [5]:
actual = actuals[1];

In [22]:
model = Chain(
    RNN(50000=>1000, tanh),
    Dense(1000=>10),
    Dense(10=>1),
    softmax
) |> gpu;

In [21]:
#Flux.reset!(model)

In [23]:
out1 = model(input |> gpu) |> cpu

1-element Vector{Float32}:
 1.0